In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

sns.set() 
#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'  # 한글 처리
plt.rcParams['axes.unicode_minus'] = False  # -, $ 등등에 표현 특수기호로 버리지 말고 다 써줘라.

In [2]:
df = pd.read_csv('./X_train.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [3]:
df.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [4]:
ydf = pd.read_csv('./y_train.csv')
ydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   cust_id  3500 non-null   int64
 1   gender   3500 non-null   int64
dtypes: int64(2)
memory usage: 54.8 KB


In [5]:
ydf.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


* 고객 3,500명에 대한 학습용 데이터를 이용해 예측모형을 구축하시오
    * 모델의 성능은 mse 평가지표에 따라 채점
    * train : 3000명 test : 500명
    * 백화점 고객의 1년 간 구매 데이터이다.
    * custid: 고객 ID
    * gender: 고객의 성별 (0: 여자, 1: 남자)

In [6]:
train        =  df.iloc[:3000, :]
train_target = ydf.iloc[:3000, :]

train.shape, train_target.shape

((3000, 10), (3000, 2))

In [7]:
test        =  df.iloc[3000:, :]
test_target = ydf.iloc[3000:, :]

test.shape, test_target.shape

((500, 10), (500, 2))

In [8]:
train        = train.drop('cust_id', axis=1)
train_target = train_target.drop('cust_id', axis=1)

test        = test.drop('cust_id', axis=1)
test_target = test_target.drop('cust_id', axis=1)


In [9]:
# 결측 처리
train['환불금액'] = train['환불금액'].fillna(0)
test['환불금액']  = test['환불금액'].fillna(0)

train['환불금액'] = train['환불금액'].astype('int')
test['환불금액']  = test['환불금액'].astype('int')

# train.info()

In [10]:
# total_train = pd.concat([train, train_target], axis=1)
# total_train.corr()

In [11]:
# test.info()

In [12]:
train['주구매상품'].nunique(), test['주구매상품'].nunique(), 

(41, 36)

In [13]:
train['주구매지점'].nunique(), test['주구매지점'].nunique(), 

(24, 22)

In [14]:
train['주구매상품'].value_counts()

기타        501
가공식품      483
농산물       294
화장품       227
시티웨어      182
디자이너      163
수산품       132
캐주얼        86
명품         81
섬유잡화       79
골프         73
스포츠        66
일용잡화       52
육류         51
모피/피혁      47
남성 캐주얼     46
구두         42
차/커피       38
건강식품       37
아동         36
피혁잡화       34
축산가공       33
주방용품       26
젓갈/반찬      25
셔츠         25
주방가전       24
트래디셔널      20
남성정장       17
생활잡화       14
주류         12
가구         10
커리어         9
대형가전        7
란제리/내의      7
액세서리        5
침구/수예       4
식기          4
보석          3
통신/컴퓨터      2
소형가전        2
남성 트랜디      1
Name: 주구매상품, dtype: int64

In [15]:
train['주구매상품'] = train['주구매상품'].replace(['남성 트랜디','남성정장','남성 캐주얼'], '남성의류')
test['주구매상품']  = test['주구매상품'].replace(['남성 트랜디','남성정장','남성 캐주얼'], '남성의류')

In [16]:
train.shape, test.shape, train_target.shape, test_target.shape

((3000, 9), (500, 9), (3000, 1), (500, 1))

In [17]:
train_test = pd.concat([train, test], axis=0)
train_test = pd.get_dummies(train_test, columns=['주구매상품','주구매지점'])

In [18]:
train_test.shape

(3500, 71)

In [19]:
train = train_test.iloc[:3000, :]
test  = train_test.iloc[3000:, :]
train.shape, test.shape

((3000, 71), (500, 71))

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(train, train_target, test_size=0.2, random_state=111)
rfc = RandomForestClassifier(random_state=111)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print(mse)
print(acc)

0.34833333333333333
0.6516666666666666


In [21]:
y_pred = rfc.predict(test)
test_mse = mean_squared_error(test_target, y_pred)
test_acc = accuracy_score(test_target, y_pred)

print(test_mse)
print(test_acc)

0.36
0.64
